In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# TensorFlow and tf.keras
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.14.0


Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# the data, split between train and test sets
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(4, 4),
                 strides=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (4, 4), strides=2, activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

W0103 17:12:43.642591 139998708766528 deprecation_wrapper.py:119] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0103 17:12:43.662852 139998708766528 deprecation_wrapper.py:119] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0103 17:12:43.665402 139998708766528 deprecation_wrapper.py:119] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

W0103 17:12:58.366697 139998708766528 deprecation_wrapper.py:119] From /usr/lib/python3/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0103 17:12:58.378581 139998708766528 deprecation_wrapper.py:119] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

W0103 17:13:01.962451 139998708766528 deprecation.py:323] From /home/jiameng/packages/Bernsp/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0103 17:13:02.069587 139998708766528 deprecation_wrapper.py:119] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 84us/step - loss: 0.9469 - acc: 0.6711 - val_loss: 0.6079 - val_acc: 0.7741
Epoch 2/12
60000/60000 [==============================] - 3s 52us/step - loss: 0.5142 - acc: 0.8097 - val_loss: 0.4840 - val_acc: 0.8268
Epoch 3/12
60000/60000 [==============================] - 3s 52us/step - loss: 0.4284 - acc: 0.8442 - val_loss: 0.4280 - val_acc: 0.8457
Epoch 4/12
60000/60000 [==============================] - 3s 52us/step - loss: 0.3854 - acc: 0.8594 - val_loss: 0.3937 - val_acc: 0.8581
Epoch 5/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.3574 - acc: 0.8699 - val_loss: 0.3834 - val_acc: 0.8603
Epoch 6/12
60000/60000 [==============================] - 3s 53us/step - loss: 0.3354 - acc: 0.8769 - val_loss: 0.3655 - val_acc: 0.8643
Epoch 7/12
 9600/60000 [===>..........................] - ETA: 2s - loss: 0.3155 - acc: 0.8839

In [11]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 1s 71us/step
Test accuracy: 0.9137


## Make predictions

In [12]:
predictions = model.predict(x_test)

In [13]:
predictions[0]

array([2.1552807e-05, 5.1608631e-06, 8.3366922e-06, 8.3626713e-07,
       2.3849253e-07, 2.8720462e-05, 5.5378507e-07, 1.5145341e-04,
       4.0970977e-07, 9.9978286e-01], dtype=float32)

In [14]:
np.argmax(predictions[0])

9

In [15]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CNNA.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CNNA.h5")
print("Saved model to disk")

Saved model to disk


In [38]:
weights = []
for layer in model.layers:
    weights.append(layer.get_weights()) # list of numpy arrays